# 卷积神经网络

作者：杨岱川

时间：2020年03月

github：https://github.com/DrDavidS/basic_Machine_Learning

开源协议：[MIT](https://github.com/DrDavidS/basic_Machine_Learning/blob/master/LICENSE)

参考文献：

- 《深度学习入门》，作者：斋藤康毅；
- 《深度学习》，作者：Ian Goodfellow 、Yoshua Bengio、Aaron Courville。
- [CS231n](http://cs231n.stanford.edu/)

## 本节目的

本节的主题就是大名鼎鼎的卷积神经网络（Convolutional Neural Network，CNN）。

CNN被广泛用于图像识别、语音识别等各种场合。在图像识别的比赛中，基于深度学习的方法几乎都以 CNN 为基础。

我们将会在本章学习 CNN 的原理和用法。

## 整体结构

首先了解 CNN 的网络结构。

CNN和之前介绍的神经网络一样，可以像积木一样通过组装各个层来构建。不过，这里出现了卷积层（Convolution层）和池化层（Pooling层），我们会在后面介绍。

这里我们先介绍如何组装层，以此来构建CNN。

回顾之前介绍的神经网络，相邻层的所有神经元之间都有连接，这叫做**全连接（fully-connected）**。我们用 Affine 层实现了全连接层。

![CNN-全连接](https://raw.githubusercontent.com/DrDavidS/basic_Machine_Learning/master/back_up_images/CNN-结构1.png)

如果使用 Affine 层构建一个 5 层的全连接神经网络，可以通过下图中的结构来实现：

![全连接结构](https://raw.githubusercontent.com/DrDavidS/basic_Machine_Learning/master/back_up_images/CNN-全连接结构.png)

如上图所示，全连接神经网络中，Affine 层后面跟的是激活函数 `ReLU`层（或者`Sigmoid`层）。第五层 Affine 层由 Softmax 输出最终结果。

那么CNN应该是什么样的呢？下图是 CNN 的一个例子：

![CNN2](https://github.com/DrDavidS/basic_Machine_Learning/blob/master/back_up_images/CNN%E7%BB%93%E6%9E%842.png?raw=true)

如图所示，CNN 中新增了 Convolution 层和 Pooling 层。一般 CNN 的层连接顺序是 “Convolution - ReLU - Pooling”，其中 Pooling 可能会被省略。

还需要注意的是，在上图的 CNN 中：

- 靠近输出的层中使用了之前的 “Affine - ReLU” 组合。

- 最后的输出层中使用了之前的 “Affine - Softmax”组合。

这些都是 CNN 中比较常见的结构。

## 卷积层

CNN 中会有各种奇奇怪怪的术语，比如 “填充” 或 “步幅” 等等。此外各层中传递的数据还是有形状的数据，比如 “3维数据”，这和之前的全连接神经网络有些不一样。

刚刚开始也许会有些难理解，不过我们会认真学习一下 CNN 中所使用的卷积层结构。

### 全连接层存在的问题

在最初的全连接神经网络中我们使用的是全连接层（Affine 层）。在全连接层中，相邻层的神经元全部连接在一起，输出的数量可以人为任意决定。

全连接层会存在什么问题呢？那就是**数据的形状被忽视了**。

比如，如果输入的数据是图像，那么图像通常是**高**、**长**、**通道**方向上的 3 维形状。但是，向全连接层输入时，需要将 3 维数据拉平为 1 维数据。

实际上在之前的 MNIST 数据集实验中，我们的输入图像就是 1 通道、高 28 像素、长 28 像素的 $(1,28,28)$ 形状，但是被我们排成了 1 列，以 784 个数据的形式输入到最开始的 Affine 层。
 
试想，图像是三维形状，这个形状里面有重要的空间信息，比如上下左右相邻的像素多为比较相近的值，RGB 各通道之间有紧密的联系；相反，较远的像素可能关联就不那么大。

但是我们采用了全连接层，会**忽略掉形状**，而将全部输入作为相同的神经元处理，所以无法利用与形状相关的信息。

> 想象一下把“图片”拉成“长条”。
 
而卷积层则可以保持形状不变。当输入数据是图像的时候，卷积层会以 3 维数据的形式接收输入数据，并以同样的 3 维数据的形式输出至下一层。因此有更高的可能性会正确理解 “图像” 等具有形状的数据。
 
此外，在 CNN 中，我们将卷积层的输入输出数据称为 **特征图（feature map）**。其中卷积层的输入数据叫做**输入特征图**，输出数据称为**输出特征图**。
 
### 卷积运算

卷积层进行的处理就是卷积运算。卷积运算相当于图像处理中的“滤波器运算”。首先我们看一个具体的例子，如下图：

![卷积运算1](https://github.com/DrDavidS/basic_Machine_Learning/blob/master/back_up_images/%E5%8D%B7%E7%A7%AF%E8%BF%90%E7%AE%971.png?raw=true)

上图中，卷及运算对输入数据应用了一个滤波器。在这个例子中，输入数据是有高、长方向形状的数据（矩形），滤波器也是如此，有高和长两个方向。

假设我们用 $(height, width)$ 表示数据和滤波器的形状，则在本例中，输入大小是 $(4, 4)$，滤波器大小是 $(3, 3)$，输出大小是 $(2, 2)$。

> 在有些文献中也会用**“核”**，或者**“卷积核”**来称呼这里所说的滤波器。

现在我们来看看卷积运算中到底发生了什么，如下面几幅图所示：

![卷积运算2](https://github.com/DrDavidS/basic_Machine_Learning/blob/master/back_up_images/%E5%8D%B7%E7%A7%AF%E8%BF%90%E7%AE%972.png?raw=true)

![卷积运算3](https://github.com/DrDavidS/basic_Machine_Learning/blob/master/back_up_images/%E5%8D%B7%E7%A7%AF%E8%BF%90%E7%AE%973.png?raw=true)

![卷积运算4](https://github.com/DrDavidS/basic_Machine_Learning/blob/master/back_up_images/%E5%8D%B7%E7%A7%AF%E8%BF%90%E7%AE%974.png?raw=true)

![卷积运算5](https://github.com/DrDavidS/basic_Machine_Learning/blob/master/back_up_images/%E5%8D%B7%E7%A7%AF%E8%BF%90%E7%AE%975.png?raw=true)

可以看到，卷积运算就是以一定的间隔滑动滤波器的窗口并应用。这里的窗口是指上面图中的灰色 $3\times 3$ 部分。

我们将各个位置上的滤波器的元素和输入的对应元素相乘，然后再求和，这个过程可以称作是**乘积累加运算**。然后，将这个结果保存到输出的对应位置。将这个过程在所有的位置都进行一遍，就可以得到卷积运算的输出。

在全连接的神经网络中，除了权重参数，还存在偏置。CNN中，滤波器的参数就对应之前的权重，而包含偏置的卷及运算的处理流程如下：

![卷积加偏置](https://github.com/DrDavidS/basic_Machine_Learning/blob/master/back_up_images/%E5%8D%B7%E7%A7%AF%E8%BF%90%E7%AE%97+%E5%81%8F%E7%BD%AE.png?raw=true)

我们在应用了滤波器（卷积核）的数据上加了偏置。偏置的大小通常是 $(1\times 1)$ 的，这个值会被加到应用了滤波器的所有元素上。

### 填充 - Padding

在进行卷积层的处理之前，有时要向输入数据的周围填入固定的数据（比如 0 ），这称为**填充（Padding）**，是卷积（以及一些文本处理）常用的方法。

在下图的例子中，对大小为 $(4\times 4)$ 的输入数据应用了幅度为 1 的填充（填充在虚线里）。

> “幅度为1的填充”是指用幅度为 1 像素的 0 填充周围。

![padding](https://github.com/DrDavidS/basic_Machine_Learning/blob/master/back_up_images/%E5%8D%B7%E7%A7%AFPADDING-%E4%BF%AE%E6%94%B9.png?raw=true)

### 步幅 - Stride